In [1]:
import requests
import json
import datetime
import time
import yaml
import FinanceDataReader as fdr
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup

In [2]:
config_path_cs = '/home/coder/project/project/stock/config.yaml'
config_path_lo = '/home/ubuntu/code-server/project/stock/config.yaml'
config_path_codespace = '/workspaces/codespaces-blank/project/stock/config.yaml'

with open(config_path_codespace, encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [3]:

def send_message(msg):
    """디스코드 메세지 전송"""
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] {str(msg)}"}
    requests.post(DISCORD_WEBHOOK_URL, data=message)
    print(message)

In [4]:
def get_access_token():
    """토큰 발급"""
    headers = {"content-type":"application/json"}
    body = {"grant_type":"client_credentials",
    "appkey":APP_KEY, 
    "appsecret":APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

In [5]:
def hashkey(datas):
    """암호화"""
    PATH = "uapi/hashkey"
    URL = f"{URL_BASE}/{PATH}"
    headers = {
    'content-Type' : 'application/json',
    'appKey' : APP_KEY,
    'appSecret' : APP_SECRET,
    }
    res = requests.post(URL, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]
    return hashkey

### test RSI module

In [9]:
df = fdr.DataReader('005930', start='2020/11/01', end='2020/12/30')
df.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-12-23,72400,74000,72300,73900,19411326,0.022130
2020-12-24,74100,78800,74000,77800,32502870,0.052774
2020-12-28,79000,80100,78200,78700,40085044,0.011568
2020-12-29,78800,78900,77300,78300,30339449,-0.005083
2020-12-30,77400,81300,77300,81000,29417421,0.034483


In [7]:
def get_rsi_ud(diff):
    ups = 0
    downs = 0

    if diff > 0:
        ups = diff
    elif diff < 0:
        downs = abs(diff)
    return pd.Series([ups, downs])

In [10]:
df['diff'] = df['Close'].diff()
df[['up', 'down']] = df['diff'].apply(get_rsi_ud)

df['au'] = df['up'].rolling(14).mean()
df['ad'] = df['down'].rolling(14).mean()

df['rs'] = df['au']/df['ad']
df['rsi'] = df['rs']/(1+df['rs'])
df['signal'] = df['rsi'].rolling(6).mean()

df.tail()

,Open,High,Low,Close,Volume,Change,diff,up,down,au,ad,rs,rsi,signal
Date,,,,,,,,,,,,,,
2020-12-23,72400,74000,72300,73900,19411326,0.022130,1600.0,1600.0,0.0,564.285714,264.285714,2.135135,0.681034,0.702519
2020-12-24,74100,78800,74000,77800,32502870,0.052774,3900.0,3900.0,0.0,714.285714,264.285714,2.702703,0.729927,0.704224
2020-12-28,79000,80100,78200,78700,40085044,0.011568,900.0,900.0,0.0,678.571429,264.285714,2.567568,0.719697,0.709359
2020-12-29,78800,78900,77300,78300,30339449,-0.005083,-400.0,0.0,400.0,678.571429,207.142857,3.275862,0.766129,0.711031
2020-12-30,77400,81300,77300,81000,29417421,0.034483,2700.0,2700.0,0.0,714.285714,207.142857,3.448276,0.775194,0.718206


In [ ]:
def get_rsi_permission(data, period_rsi=14, period_signal=6, col='Close'):
    df = data

    df['diff'] = df[col].diff()
    df[['up', 'down']] = df['diff'].apply(get_rsi_ud)

    df['au'] = df['up'].rolling(period_rsi).mean()
    df['ad'] = df['down'].rolling(period_rsi).mean()
    
    df['rs'] = df['au']/df['ad']
    df['rsi'] = df['rs']/(1+df['rs'])
    df['signal'] = df['rsi'].rolling(period_signal).mean()
    rsi = float(df['rsi'].tail()[1])
    signal = float(df['signal'].tail()[1])

    if rsi<=0.35:
        return 'long'
    elif rsi>=0.65:
        return 'short'
    else:
        return 'don'

### RSI module 그대로 사용  
<br/>  

- - -  

<br/>  

### test MACD moudule

In [ ]:
def get_simpleMovingAverage(data, period=10, col='Close'):
    data['sma_'+str(period)] = data[col].rolling(period).mean()

    return float(data['sma_'+str(period)].tail()[4])

def get_exponentialMovingAverage(data, period=7, col='Close'):
    data['ema_'+str(period)] = data[col].ewm(span=period, adjust=False).mean()

    return float(data['ema_'+str(period)].tail()[4])

def get_macd_permission(data, period_long=26, period_short=12, period_signal=9, col='Close'):
    shortEMA = get_exponentialMovingAverage(data, period_short, col=col)
    longEMA = get_exponentialMovingAverage(data, period_long, col=col)
    data['macd'] = shortEMA - longEMA
    signalEMA = get_exponentialMovingAverage(data, period_signal, col='macd')
    macd = shortEMA - longEMA
    signal = signalEMA

    if macd>=signal:
        return 'long'
    elif macd<signal:
        return 'short'